In [1]:
! pwd

/Users/sarahwooller/Sussex/CRUK/vision/CRUK_datahub_landing_page/python_helper_code_and_jsons


In [50]:
import os
import json

In [51]:
path = "../src/utils/longer_filter_data.js"

In [52]:
with open(path) as f:
    text = f.read().split("\n")

In [53]:
dictionary = eval(text[1][:-1])

In [54]:
with open("../src/utils/data.json","w") as f:
    json.dump(dictionary, f)

In [24]:


def get_labels(dictionary):
    labels=[]
    for key in dictionary.keys():
        d = dictionary[key]
        labels.append(d['label'])
        if 'children' in d.keys():
           labels += get_labels(d['children'])
    return labels

In [25]:
top = dictionary["0_0"]['children']["0_0_0"]['children']
top_labels = get_labels(top)

In [32]:
hist = dictionary["0_0"]['children']["0_0_1"]['children']
hist_labels = get_labels(hist)

In [33]:
tcga_labels = get_labels(dictionary["0_0"]['children']["0_0_4"]['children'])

In [36]:
cruk_labels = get_labels(dictionary["0_0"]['children']["0_0_2"]['children'])

In [39]:
cruk_labels = cruk_labels[1:]

In [40]:
labels = {"ICD-O topography": top_labels, 
          "ICD-O histology": hist_labels,
         "CRUK": cruk_labels, 
         "TCGA":tcga_labels}

In [49]:
import json
with open("../src/utils/labels.json", "w") as f:
    json.dump(labels, f)

In [44]:
[i for i in labels["ICD-O topography"] if 'reast' in i]

['C50 Breast',
 'C50.1 Central portion of breast',
 'C50.2 Upper-inner quadrant of breast',
 'C50.3 Lower-inner quadrant of breast',
 'C50.4 Upper-outer quadrant of breast',
 'C50.5 Lower-outer quadrant of breast',
 'C50.6 Axillary tail of breast',
 'C50.8 Overlapping lesion of breast',
 'C50.9 Breast, nos']

Please can you make me a json dictionary  which takes each of the  CRUK terms in turn and returns a dictionary with a list of the appropriate "ICD-O topography" and another list of the appropriate "ICD-O histology" terms, and then does the same for the TCGA terms.
The format will therefore look like this :

{"CRUK":{'Acute lymphoblastic leukaemia (ALL)':{"ICD-O topography":[terms here], "ICD-O histology":[terms here], }...},
"TCGA":{'ACC (Adrenocortical carcinoma)':{"ICD-O topography":[terms here], "ICD-O histology":[terms here], }...},}

The terms should be drawn exclusively from the "ICD-O topography" and "ICD-O histology" lists given with no changes to formatting or spelling (so that it can be computer read)

The highest level term should be preferred: ie Breast Cancer would return just ['C50 Breast']
No extra words please


In [12]:
from typing import Dict, Any, List, Optional

class TreeNode:
    """
    Represents a single node in the hierarchical tree structure.
    """
    def __init__(
        self,
        id: str,
        label: str,
        category: str,
        primaryGroup: str,
        count: str,
        description: str,
        children: Optional[List['TreeNode']] = None,
    ):
        """
        Initializes a new tree node with data attributes.

        Args:
            id (str): The unique identifier for the node.
            label (str): The human-readable label or name.
            category (str): The category of the node.
            primary_group (str): The primary group classification.
            count (str): A count associated with the node.
            children (Optional[List['TreeNode']]): A list of child TreeNode objects.
        """
        self.id = id
        self.label = label
        self.category = category
        self.primaryGroup = primaryGroup
        self.description = description
        self.count = count
        self.children: List['TreeNode'] = children if children is not None else []

    def __repr__(self):
        """
        Provides a string representation for debugging.
        """
        return f"TreeNode(id='{self.id}', label='{self.label}', children={len(self.children)})"

    def print_tree(self, level=0):
        """
        Recursively prints the structure of the tree.
        """
        indent = "  " * level
        print(f"{indent}- {self.label} (ID: {self.id})")
        for child in self.children:
            child.print_tree(level + 1)

In [20]:
def build_dict(tree: TreeNode) -> dict:
    "Recursively builds the new dictionary from the tree"

    dictionary = {}
    for label in ["id", "label", "category", "primaryGroup", "count"]:
        dictionary[label] = getattr(tree, label)
    if tree.children:
        dictionary["children"] = dict([(child.id, build_dict(child)) for child in tree.children])
    return dictionary
    

In [14]:
def build_tree(data: Dict[str, Any]) -> TreeNode:
    """
    Recursively builds the TreeNode structure from the raw dictionary data.

    Args:
        data (Dict[str, Any]): The raw dictionary data for a single node.

    Returns:
        TreeNode: The resulting object-oriented tree node.
    """
    # Extract core attributes
    node_id = data.get('id', '')
    label = data.get('label', '')
    category = data.get('category', '')
    primaryGroup = 'cancer-type'
    count = data.get('count', '')
    description = data.get('description', '')

    # Initialize the current node
    current_node = TreeNode(
        id=node_id,
        label=label,
        category=category,
        primaryGroup=primaryGroup,
        count=count,
        description=description
    )

    # Check for children and recurse
    raw_children = data.get('children', {})
    if raw_children and isinstance(raw_children, dict):
        # Iterate over the values (the child dictionaries)
        for child_dict in raw_children.values():
            # Recursively build the child node
            child_node = build_tree(child_dict)
            current_node.children.append(child_node)

    return current_node

In [15]:
tree = build_tree(dictionary['0_0'])

In [16]:
tcga = tree.children.pop()

In [17]:
for child in tcga.children:
    child.label = f"{child.label} ({child.description})"

In [18]:
tree.children.append(tcga)

In [21]:
new_dictionary = build_dict(tree)

In [22]:
dictionary['0_0'] = new_dictionary

In [23]:
path = "../src/utils/longer_filter_data.js"

In [24]:
with open(path, "w") as f:
    f.write(text[0])
    f.write("\n")

In [25]:
with open(path, "a") as f:
    json.dump(dictionary, f)

In [26]:
with open(path, "a") as f:
    f.write(";\n")
    f.write(text[2])

In [27]:
with open("../src/utils/lfd.json","w") as f:
    json.dump(dictionary, f)